In [16]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [17]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.optim.lr_scheduler import MultiStepLR

In [18]:
import pandas as pd
import numpy as np
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import time
from torch.autograd import Variable
use_cuda = torch.cuda.is_available()

In [19]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [20]:
def get_loader(batch_size=128,num_workers=4,mixup=True,alpha=1):
    collator = torch.utils.data.dataloader.default_collate

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_dataset = datasets.CIFAR10(root='cifar', train=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize,]), download=True)
      
    dev_dataset=datasets.CIFAR10(root='cifar', train=False, transform=transforms.Compose([                                                                    
                transforms.ToTensor(),
                normalize,]))

    train_dataloader = torch.utils.data.DataLoader(train_dataset,      
                       batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, pin_memory=True,
                       drop_last=True,collate_fn=collator)
    
    dev_dataloader = torch.utils.data.DataLoader( dev_dataset, 
                     batch_size=batch_size, shuffle=False,
                     num_workers=num_workers, pin_memory=True,
                     drop_last=False)
    return train_dataloader,dev_dataloader

In [21]:
train_dataloader,dev_dataloader=get_loader(batch_size=128,num_workers=4,mixup=True,alpha=1)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## model

In [22]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out



## training

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ResNet(BasicBlock, [2,2,2,2], 10)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [24]:
criterion = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss(size_average=True)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,200,0)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [25]:
trainingloss=[]
trainingacc=[]
valloss=[]
valacc=[]
valacc5=[]


In [26]:
epoch=200
best_accuracy=0.0
es=0
correct=0
start_time=time.time()
for i in range(epoch):
    model.train()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_data_num = len(train_dataloader.dataset)
    steps = 0.0
    #训练
    for batch in train_dataloader:
        steps+=1
        optimizer.zero_grad() 
        # 取数据
        inputs, targets = batch
        inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, use_cuda)
        inputs, targets_a, targets_b = map(Variable, (inputs,targets_a, targets_b))
        outputs = model(inputs)
        loss = mixup_criterion(criterion,outputs,targets_a,targets_b,lam)
        loss.backward()
        optimizer.step()  

        total_loss = total_loss + loss.item() 
#计算正确率
        _, predicted = torch.max(outputs.data, 1)

        
        correct += predicted.eq(targets.data).cpu().sum()
        
        
        if steps%100==0:
          print("Epoch %d_%.3f%%:  Training average Loss: %f"
                      %(i, steps * train_dataloader.batch_size*100/len(train_dataloader.dataset),total_loss/steps))
    print("Epoch %d :  Training average Loss: %f, Training accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, correct*100/total_data_num,time.time()-start_time))   
    trainingloss.append(total_loss/steps)
    trainingacc.append(total_correct/total_data_num) 
    
    
    
    scheduler.step()
    
       #验证
    model.eval()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_correctk=0.0
    total_data_num = len(dev_dataloader.dataset)
    steps = 0.0    
    for batch in dev_dataloader:
        steps+=1
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)  # 将输入和目标在每一步都送入GPU
        outputs = model(inputs)
        loss = criterion2(outputs, labels)  
        total_loss = total_loss + loss.item() 
        correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(labels.size()) == labels).sum()
        total_correct = total_correct + correct.item()

        maxk = max((1,5))
        yresize = labels.view(-1,1)
        _, pred = outputs.topk(maxk, 1, True, True)

        correctk = torch.eq(pred, yresize).sum()

        #correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   #.view(labels.size()) == labels).sum()
        total_correctk = total_correctk + correctk.item()
        
    print("Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%,Verification 5 accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, total_correct*100/total_data_num,total_correctk*100/total_data_num,time.time()-start_time))  
    #print("Epoch %d :  Verification 5 accuracy: %f%%,Total Time:%f"
      #%(i,  total_correctk*100/total_data_num,time.time()-start_time))  
    valloss.append(total_loss/steps)
    valacc.append(total_correct/total_data_num) 
    valacc5.append(total_correctk/total_data_num) 
    if best_accuracy < total_correct/total_data_num :
        es = 0
        best_accuracy =total_correct/total_data_num 
        torch.save(model,'/content/mydrive/MyDrive/project/outputs/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        print('Model is saved in /content/mydrive/MyDrive/project/outputs/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        #torch.cuda.empty_cache()
    else:
        es += 1
        print("Counter {} ".format(es))
        

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0_25.600%:  Training average Loss: 3.102813
Epoch 0_51.200%:  Training average Loss: 2.700293
Epoch 0_76.800%:  Training average Loss: 2.556320
Epoch 0 :  Training average Loss: 2.471508, Training accuracy: 12.166000%,Total Time:18.194915
Epoch 0 :  Verification average Loss: 2.069221, Verification accuracy: 21.560000%,Verification 5 accuracy: 75.380000%,Total Time:20.363530
Model is saved in /content/mydrive/MyDrive/project/outputs/epoch_0_accuracy_0.215600
Epoch 1_25.600%:  Training average Loss: 2.142261
Epoch 1_51.200%:  Training average Loss: 2.126488
Epoch 1_76.800%:  Training average Loss: 2.115135
Epoch 1 :  Training average Loss: 2.103573, Training accuracy: 18.205999%,Total Time:38.814796
Epoch 1 :  Verification average Loss: 1.925677, Verification accuracy: 28.890000%,Verification 5 accuracy: 80.800000%,Total Time:40.887495
Model is saved in /content/mydrive/MyDrive/project/outputs/epoch_1_accuracy_0.288900
Epoch 2_25.600%:  Training average Loss: 2.026596
Epoch 2_51.2

In [27]:
trainingloss=pd.DataFrame(data=trainingloss)
trainingloss.to_csv('/content/mydrive/MyDrive/project/outputs/exp2/trainingloss.csv',encoding='utf-8')

trainingacc=pd.DataFrame(data=trainingacc)
trainingacc.to_csv('/content/mydrive/MyDrive/project/outputs/exp2/trainingacc.csv',encoding='utf-8')

valloss=pd.DataFrame(data=valloss)
valloss.to_csv('/content/mydrive/MyDrive/project/outputs/exp2/valloss.csv',encoding='utf-8')

valacc=pd.DataFrame(data=valacc)
valacc.to_csv('/content/mydrive/MyDrive/project/outputs/exp2/valacc.csv',encoding='utf-8')

valacc5=pd.DataFrame(data=valacc5)
valacc5.to_csv('/content/mydrive/MyDrive/project/outputs/exp2/valacc5.csv',encoding='utf-8')